In [80]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

import scipy.constants as scc

import sys
sys.path.append('../RD-systems-and-test-benches')

import thermal.SK_standard as nm

import seaborn as sns

import utils.plot_functions as RD_pfun
import repo_config as rconfig

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import plotly.io as pio

from itertools import product
from itertools import chain

from tqdm.notebook import tqdm

import copy

from concurrent.futures import ProcessPoolExecutor

In [499]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.1I.xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\100424-tests-conditions-GMI.xlsx'
exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

# panelSpecs = pr.import_geometry(geometry_path)
# hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

In [82]:
recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

for key, group in groups.items():
    pr.convert_to_K(group)

In [83]:
triplets = [
            {'design_name' : 'V4.1NI',
             'test_type' : 'SK'},
            {'design_name' : 'V4.1NI',
                'test_type' : 'dark'},
            {'design_name' : 'V4.1I',
                'test_type' : 'SK'},
            {'design_name' : 'V4.1I',
                'test_type' : 'dark'}
                ]

for triplet in triplets:
             
    triplet['panelSpecs'] = pr.import_geometry(os.path.join(f'{folder_path}\\Panels', f'Panel_V4.1I.xlsx'))

    if triplet['design_name'] == 'V4.1NI':
        triplet['panelSpecs']['part2']['lambd_abs_ins'] = 0.
        triplet['panelSpecs']['part2']['lambd_tube_ins'] = 0.

        pr.update_panelSpecs_after_modif(triplet['panelSpecs'])
        
    triplet['hyp'] = pr.create_dict_from_excel(hypotheses_path,'Main')
    triplet['hyp']['method_h_top_g_exchanger'] = "forced_with_coeff_steps"
    triplet['steadyStateConditions_df'] = groups[triplet['design_name']].loc[groups[triplet['design_name']]['Type'] == triplet['test_type']].copy()
    pr.adjust_steadyStateConditions(triplet['steadyStateConditions_df'],triplet['hyp'])
    triplet['steadyStateConditions_df'].reset_index(drop=True,inplace=True)

### Implementation

In [ ]:
0.9*0.87

In [84]:
mapping_parameters_bounds = {

    'V4.1NI' : {
        'panelSpecs': {
            'meander' : {
                    'lambd_air' : (0.05e-3,2.e-3),
            }
        }
    },

    'V4.1I' : {
        'panelSpecs': {
            'meander' : {
                    'lambd_air' : (0.05e-3,2.e-3),
            }
        }
    },

    'shared_designs' : {
        'panelSpecs': {
            'pv' : {
                    'tau_g' : (0.87, 0.93),
                    'alpha_g' : (0.03,0.09),
                    # 'alpha_PV' : (0.87, 0.93),
                    'eps_PV' : (0.85, 0.95),
                    'eps_g' : (0.85, 0.95),
            },
            'meander' : {
                    'l_c' : (0.1e-3,0.6e-3),
                    'eps_tube' : (0.01,0.20),
                    'eps_ins' : (0.25,0.60),
            },
        }
    },

    'hyp': {
        'coeff_h_top_forced_range0' : (0.5, 3.),
        'coeff_h_top_forced_range1' : (0.5, 3.),
        'coeff_h_top_forced_range2' : (0.5, 3.),
    },
}

# Function to recursively flatten the parameter bounds
def flatten_parameters_bounds(d, parent_keys=[]):
    items = []
    for k, v in d.items():
        if isinstance(v, dict):
            items.extend(flatten_parameters_bounds(v, parent_keys + [k]))
        else:
            items.append((parent_keys + [k], v))
    return items

# Flatten the parameter bounds and extract parameter paths and bounds
flattened_parameters = flatten_parameters_bounds(mapping_parameters_bounds)
parameter_paths = [param[0] for param in flattened_parameters]
bounds = [param[1] for param in flattened_parameters]

mapping_parameters = {tuple(flattened_parameters[j][0]): j for j in range(len(flattened_parameters))}

In [85]:
def flatten_dict(d, parent_key=()):
    """ Recursively flattens a nested dictionary. Keys become tuples of the path."""
    items = []
    for k, v in d.items():
        new_key = parent_key + (k,)
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [86]:
def update_specs(params, design, panelSpecs):
    for key in mapping_parameters_bounds[design]['panelSpecs']:
        if key in list(set(panelSpecs['decomp'].values())) + ['pv']:
            zones = list(panelSpecs['decomp'].keys()) + ['pv'] if key == 'pv' else [_ for _, value in panelSpecs['decomp'].items() if value == key]
            for z in zones:
                for subkey in mapping_parameters_bounds[design]['panelSpecs'][key]:
                    panelSpecs[z][subkey] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]

                    if subkey == 'eps_tube':
                        panelSpecs['part1']['eps_tube'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
                        panelSpecs['part3']['eps_tube'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
        else:
            panelSpecs[key] = params[mapping_parameters[(design, 'panelSpecs', key)]]

def update_panelSpecs(params, design_name, panelSpecs):
    update_specs(params, design_name, panelSpecs)
    update_specs(params, 'shared_designs', panelSpecs)
    pr.update_panelSpecs_after_modif(panelSpecs)

def update_hyp(params, hyp):

    for key in mapping_parameters_bounds['hyp']:
        hyp[key] = params[mapping_parameters[('hyp', key)]]

In [87]:
def simulation_function(params, triplet):

    panelSpecs = copy.deepcopy(triplet['panelSpecs'])
    hyp = copy.deepcopy(triplet['hyp'])
    steadyStateConditions_df = triplet['steadyStateConditions_df']

    update_panelSpecs(params, triplet['design_name'], panelSpecs)
    update_hyp(params, hyp)

    # Call the simulation function
    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)
    return df_res


# Global counter for objective function calls
call_count = 0

# def callback_function(xk):
#     print(f"Iteration {len(iteration_history)}: params = {xk}")
#     iteration_history.append(xk.copy())

def objective_function(params, triplets):

    global call_count
    call_count += 1

    total_error = 0.0

    for triplet in triplets:

        df_res = simulation_function(params, triplet)

        # Compute the error
        error = np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean())
        total_error += error

    return total_error

def run_optimization(initial_guess, bounds, triplets):
    result = sco.minimize(
        objective_function,
        x0=initial_guess,
        args=(triplets,),  # Ensure you're passing the right triplet or list of triplets
        bounds=bounds,
        method='L-BFGS-B',
        options={'disp': True}
    )
    return result

### Differentiel evolution with intermediate solutions

In [88]:
params_init = np.np.array([9.97600358e-04, 5.31001503e-04, 9.0e-01, 7.38124530e-02,
        9.0e-01, 9.0e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00])

In [117]:
popsize = 15

initial_population = np.tile(params_init, (popsize, 1))

# Add small random perturbations to params_init
perturbation_strength = 0.00001  # Change this to control how much variation you want
initial_population = params_init + perturbation_strength * np.random.randn(popsize, len(params_init))

In [ ]:
# Check if all elements of the initial population respect the bounds
def check_bounds(population, bounds):
    for individual in population:
        for value, (lower, upper) in zip(individual, bounds):
            if not (lower <= value <= upper):
                return False
    return True

# Check the initial population
is_within_bounds = check_bounds(initial_population, bounds)
print(f"All elements of the initial population respect the bounds: {is_within_bounds}")

In [ ]:
initial_population.shape

In [120]:
# Define a callback function to store intermediate results
intermediate_solutions = []

def callback(xk, convergence):
    intermediate_solutions.append(xk)

result = sco.differential_evolution(
    objective_function,
    bounds=bounds,
    args=(triplets,),
    strategy='best1bin',
    maxiter=500,
    popsize=15,
    tol=0.01,
    callback=callback,
    polish=False,  # Set to False to prevent local search polishing
    init=initial_population
)

In [137]:
intermediate_solutions = [np.array([1.05486699e-03, 4.43340782e-04, 9.00004223e-01, 7.38234524e-02,
        9.00023522e-01, 9.00024518e-01, 2.98474858e-04, 8.46039482e-02,
        3.68719263e-01, 1.15642351e+00, 1.63743897e+00, 1.96086517e+00]),
 np.array([1.02382304e-03, 4.56092331e-04, 8.99998127e-01, 7.38462285e-02,
        9.00010041e-01, 9.00010428e-01, 2.90384600e-04, 8.45375396e-02,
        3.68655297e-01, 1.15642177e+00, 1.63744559e+00, 1.96086877e+00]),
 np.array([1.01474095e-03, 4.33272319e-04, 9.00004888e-01, 7.38621487e-02,
        9.00006180e-01, 9.00014558e-01, 2.89944614e-04, 8.45770935e-02,
        3.68576150e-01, 1.15642096e+00, 1.63739459e+00, 1.96086865e+00]),
 np.array([9.82412845e-04, 4.53903921e-04, 9.00000292e-01, 7.37756753e-02,
        9.00021063e-01, 9.00033293e-01, 3.30863768e-04, 8.45245531e-02,
        3.68907630e-01, 1.15642886e+00, 1.63744446e+00, 1.96090687e+00]),
 np.array([9.76508324e-04, 4.37208086e-04, 8.99994098e-01, 7.36639707e-02,
        9.00037102e-01, 9.00005771e-01, 2.93269678e-04, 8.45218574e-02,
        3.68436426e-01, 1.15638749e+00, 1.63746380e+00, 1.96092675e+00]),
 np.array([9.76508324e-04, 4.37208086e-04, 8.99994098e-01, 7.36639707e-02,
        9.00037102e-01, 9.00005771e-01, 2.93269678e-04, 8.45218574e-02,
        3.68436426e-01, 1.15638749e+00, 1.63746380e+00, 1.96092675e+00]),
 np.array([9.76508324e-04, 4.37208086e-04, 8.99994098e-01, 7.36639707e-02,
        9.00006180e-01, 8.99995088e-01, 3.00029029e-04, 8.45770935e-02,
        3.67927948e-01, 1.15638749e+00, 1.63746380e+00, 1.96087442e+00])]

In [124]:
params_minima = [{tuple(param[0]): result['x'][i] for i, param in enumerate(flattened_parameters)}]

In [ ]:
params_minima

In [ ]:
params_init

## Analysis of local minima

In [71]:
intermediate_solutions_prev = [
 np.array([1.22640918e-03, 3.98644180e-04, 9.18899094e-01, 6.85987125e-02,
        8.71481814e-01, 8.51242263e-01, 9.30423518e-01, 2.17231649e-04,
        1.65443828e-02, 5.82854665e-01, 1.25041063e+00, 1.20041007e+00,
        1.93000593e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.26217445e-04, 4.37677283e-04, 8.41113150e-01, 5.38871927e-02,
        8.91305951e-01, 8.74632721e-01, 9.39841780e-01, 1.36136866e-03,
        9.12009825e-02, 5.74203940e-01, 5.98633964e-01, 1.12099115e+00,
        1.56346626e+00]),
 np.array([8.62018995e-04, 3.13082835e-04, 8.14075138e-01, 5.55493487e-02,
        8.91496533e-01, 8.62556620e-01, 9.21293925e-01, 7.95060043e-04,
        3.66820264e-02, 5.63992513e-01, 5.24918043e-01, 1.07854339e+00,
        1.62695967e+00]),
 np.array([8.62018995e-04, 3.13082835e-04, 8.14075138e-01, 5.55493487e-02,
        8.91496533e-01, 8.62556620e-01, 9.21293925e-01, 7.95060043e-04,
        3.66820264e-02, 5.63992513e-01, 5.24918043e-01, 1.07854339e+00,
        1.62695967e+00]),
 np.array([8.62018995e-04, 3.13082835e-04, 8.14075138e-01, 5.55493487e-02,
        8.91496533e-01, 8.62556620e-01, 9.21293925e-01, 7.95060043e-04,
        3.66820264e-02, 5.63992513e-01, 5.24918043e-01, 1.07854339e+00,
        1.62695967e+00]),
 np.array([8.62018995e-04, 3.13082835e-04, 8.14075138e-01, 5.55493487e-02,
        8.91496533e-01, 8.62556620e-01, 9.21293925e-01, 7.95060043e-04,
        3.66820264e-02, 5.63992513e-01, 5.24918043e-01, 1.07854339e+00,
        1.62695967e+00]),
 np.array([8.62018995e-04, 3.13082835e-04, 8.14075138e-01, 5.55493487e-02,
        8.91496533e-01, 8.62556620e-01, 9.21293925e-01, 7.95060043e-04,
        3.66820264e-02, 5.63992513e-01, 5.24918043e-01, 1.07854339e+00,
        1.62695967e+00]),
 np.array([8.62018995e-04, 3.13082835e-04, 8.14075138e-01, 5.55493487e-02,
        8.91496533e-01, 8.62556620e-01, 9.21293925e-01, 7.95060043e-04,
        3.66820264e-02, 5.63992513e-01, 5.24918043e-01, 1.07854339e+00,
        1.62695967e+00]),
 np.array([9.43532469e-04, 3.53907719e-04, 8.03672335e-01, 6.94230471e-02,
        9.22619118e-01, 8.77030239e-01, 9.27660311e-01, 8.44245345e-04,
        8.70672478e-02, 5.02834649e-01, 5.73639033e-01, 1.14610715e+00,
        1.92455340e+00]),
 np.array([9.43532469e-04, 3.53907719e-04, 8.03672335e-01, 6.94230471e-02,
        9.22619118e-01, 8.77030239e-01, 9.27660311e-01, 8.44245345e-04,
        8.70672478e-02, 5.02834649e-01, 5.73639033e-01, 1.14610715e+00,
        1.92455340e+00]),
 np.array([8.68680576e-04, 3.55281056e-04, 8.30851952e-01, 6.09084196e-02,
        8.79561464e-01, 8.65271481e-01, 8.98614138e-01, 7.55104678e-04,
        1.50371346e-01, 5.01130347e-01, 5.77305374e-01, 9.72626352e-01,
        1.72365520e+00]),
 np.array([8.84102575e-04, 3.61236252e-04, 8.05773421e-01, 5.74176723e-02,
        9.17113287e-01, 8.66488799e-01, 9.07974893e-01, 5.15141965e-04,
        1.85864838e-01, 4.40199802e-01, 5.19046529e-01, 1.11480520e+00,
        1.80035857e+00]),
 np.array([8.84102575e-04, 3.61236252e-04, 8.05773421e-01, 5.74176723e-02,
        9.17113287e-01, 8.66488799e-01, 9.07974893e-01, 5.15141965e-04,
        1.85864838e-01, 4.40199802e-01, 5.19046529e-01, 1.11480520e+00,
        1.80035857e+00]),
 np.array([8.67318812e-04, 3.64902716e-04, 8.19296641e-01, 5.32499323e-02,
        8.98055255e-01, 8.69559294e-01, 8.76879875e-01, 1.01696471e-03,
        1.42209079e-01, 3.31983955e-01, 5.42409113e-01, 1.07001097e+00,
        1.84805530e+00]),
 np.array([8.40209497e-04, 3.18418839e-04, 8.12839958e-01, 5.40984779e-02,
        8.90587238e-01, 8.66545054e-01, 8.60366531e-01, 9.22564041e-04,
        1.71156053e-01, 4.01158247e-01, 5.39571853e-01, 1.09984785e+00,
        1.72535809e+00]),
 np.array([8.40209497e-04, 3.18418839e-04, 8.12839958e-01, 5.40984779e-02,
        8.81827063e-01, 8.69277497e-01, 8.67205916e-01, 1.03633288e-03,
        1.91674103e-01, 3.58707400e-01, 5.37391993e-01, 1.07192085e+00,
        1.72535809e+00]),
 np.array([8.40209497e-04, 3.18418839e-04, 8.12839958e-01, 5.40984779e-02,
        8.81827063e-01, 8.69277497e-01, 8.67205916e-01, 1.03633288e-03,
        1.91674103e-01, 3.58707400e-01, 5.37391993e-01, 1.07192085e+00,
        1.72535809e+00])]

In [18]:
intermediate_solutions_filtered = [x for x in intermediate_solutions if x[10] >= 1]

In [131]:
intermediate_solutions_filtered = [np.array([1.22640918e-03, 3.98644180e-04, 9.18899094e-01, 6.85987125e-02,
        8.71481814e-01, 8.51242263e-01, 9.30423518e-01, 2.17231649e-04,
        1.65443828e-02, 5.82854665e-01, 1.25041063e+00, 1.20041007e+00,
        1.93000593e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00])]

intermediate_solutions_filtered = intermediate_solutions_prev[:4]

In [26]:
T_s = 35 + 273.15
T_amb = 18 + 273.15
longueur = 1.708

In [27]:
dfu_dic = {}

T_s_range = np.arange(16, )

    dfu = pd.DataFrame(np.linspace(0.1,4.,10),columns=['u'])
    dfu['lam+turb'] = dfu.apply(lambda x : bht.h_top_forced(T_s,T_amb,x['u'],longueur),axis=1)
    dfu['turb'] = dfu.apply(lambda x : bht.h_top_forced_turbulent(T_s,T_amb,x['u'],longueur),axis=1)
    dfu['ratio'] = dfu['turb']/dfu['lam+turb']

In [141]:
prev_solutions = [np.array([1.22640918e-03, 3.98644180e-04, 9.18899094e-01, 6.85987125e-02,
        8.71481814e-01, 8.51242263e-01, 9.30423518e-01, 2.17231649e-04,
        1.65443828e-02, 5.82854665e-01, 1.25041063e+00, 1.20041007e+00,
        1.93000593e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00]),
 np.array([9.97600358e-04, 5.31001503e-04, 9.41048703e-01, 7.38124530e-02,
        8.75650753e-01, 8.69337111e-01, 9.22017187e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00])
    ]

In [149]:
prev_solutions_2 = []

for params in prev_solutions:
    tau_g = params[2] * params[4] / 0.9
    prev_solutions_2.append(np.array([params[0], params[1], tau_g, params[3], params[5], params[6], params[7], params[8], params[9], params[10], params[11], params[12]]))

In [ ]:
prev_solutions_2 + intermediate_solutions

In [ ]:
u_range = [0.8, 1.6, 2.8]

for i in range(len(intermediate_solutions)):
    plt.scatter(u_range, intermediate_solutions[i][-3:], label=f"Intermediate solution {i}")

# plt.plot(dfu['u'],dfu['lam+turb'],label='Laminar + Turbulent')
# plt.plot(dfu['u'],dfu['turb'],label='Turbulent')

plt.plot(dfu['u'],dfu['ratio'],label='Theoretical ratio')
# plt.legend()

In [ ]:
flattened_parameters

In [ ]:
[(flattened_parameters[i][0], intermediate_solutions[4][i]) for i in range(len(intermediate_solutions[-1]))]

In [ ]:
0.88/0.9

In [ ]:
0.94*0.8756

In [ ]:
intermediate_solutions[-1]

In [ ]:
prev_solutions_2 + intermediate_solutions

In [ ]:
prev_solutions_2 + intermediate_solutions

In [151]:
df_res_minima = []
mae_minima = []
rmse_minima = []

for params_minimum in prev_solutions_2 + intermediate_solutions:
    mae_minima.append([])
    rmse_minima.append([])

    df_concat = pd.DataFrame()
    for triplet in triplets:
        df_res = simulation_function(params_minimum, triplet)

        triplet['df_res'] = df_res

        df_res['Qdot_experimental'] = triplet['steadyStateConditions_df']['Qdot']
        df_concat = pd.concat([df_concat, df_res], axis=0)
        mae_minima[-1].append(sklearn.metrics.mean_absolute_error(df_res['Qdot_tube_fluid'], df_res['Qdot_experimental']))
        rmse_minima[-1].append(np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean()))
    df_res_minima.append(df_concat)

In [ ]:
mae_minima[4]

In [155]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

In [ ]:
rconfig.get_hex_code('red', 100)

In [183]:
design_names = ['S-S\&T-NI',
                'S-S\&T-NI',
                'S-S\&T-I',
                'S-S\&T-I']

In [ ]:
design_names[0]

In [ ]:
df_res_minima[-1]

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,3)]
k = 0

print(sklearn.metrics.mean_absolute_error(triplets[k]['df_res']['Qdot_tube_fluid'], triplets[k]['df_res']['Qdot_experimental']))
for u_min, u_max in u_range:

    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model')
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color='red')

# ax.spines['left'].set_position('zero')  # y-axis at x = 0

In [ ]:
colors

In [ ]:
triplets[0]['design_name']

Figure a

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,3)]
k = 0

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

# ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
# ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

ax.tick_params(axis='x', which='major', pad=35)
# ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
              labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-NI - SK',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [ ]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-1-fig1a.png', dpi=600)

Figure b

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,3)]
k = 1

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
# ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

# ax.tick_params(axis='x', which='major', pad=35)
ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
              labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-NI - dark',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [408]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-1-fig1b.png', dpi=600)

Figure c

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,3)]
k = 2

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

# ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

# ax.tick_params(axis='x', which='major', pad=35)
# ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
            #   labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-I - SK',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [401]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-1-fig1c.png', dpi=600)

Figure d

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,3)]
k = 3

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
# ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

# ax.tick_params(axis='x', which='major', pad=35)
ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
              labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-I - dark',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [410]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-1-fig1d.png', dpi=600)

In [428]:
markers = ['o', 's', 'd', '^', 'P', 'v']

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

colors = [rconfig.get_hex_code('red', 150),
          rconfig.get_hex_code('olive', 100),
          rconfig.get_hex_code('orange', 100),
          rconfig.get_hex_code('blue', 150),]

for j in range(0,4):
    print(str(design_names[j]) + ' ' + triplets[j]['test_type'])
    ax.plot(np.arange(0,len(mae_minima),1), [mae_minima[i][j] for i in range(len(mae_minima))], label= str(design_names[j]) + ' - ' + triplets[j]['test_type'], marker=markers[j], linewidth = 1.5, color = colors[j], linestyle = linestyles[j])
# ax.plot(np.arange(0,len(mae_minima),1), [np.array([mae_minima[i][j] for j in range(0,4)]).sum() for i in range(len(mae_minima))], label='MAE')

ax.set_xlabel('Local minimum number')
ax.set_ylabel('MAE (W)')
ax.grid(True, linewidth=0.5, color=rconfig.get_hex_code('grey', 70))
ax.set_ylim(0,37)
ax.legend(handlelength=4,  # Increase length of line in legend
           handleheight=0.8,   # Increase height of marker in legend
           borderpad=1.5,    # Padding between legend box and text
           labelspacing=0.8, # Space between legend entries
           handletextpad=1,  # Space between line and text
           loc='upper right',# Position the legend
           fontsize='medium')  # Adjust font size

fig.show()

In [441]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_TH\TH-1-fig2.png', dpi=600)

In [ ]:
mae_minima

In [384]:
global_min = np.array([9.76508324e-04, 4.37208086e-04, 8.99994098e-01, 7.36639707e-02,
        9.00006180e-01, 8.99995088e-01, 3.00029029e-04, 8.45770935e-02,
        3.67927948e-01, 1.15638749e+00, 1.63746380e+00, 1.96087442e+00])

In [385]:
global_min_with_index = [{tuple(param[0]): global_min[i] for i, param in enumerate(flattened_parameters)}]

In [ ]:
global_min_with_index

In [ ]:
0.000300029029*1000

In [ ]:
0.9*0.9

In [449]:
dic_names_exp = {'T_in' : 'T_fluid_in0',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

dic_names_num = {'T_in' : 'T_fluid_in',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

In [463]:
for _ in range(4):
    triplets[_]['steadyStateConditions_df']['Qdot/AG'] = triplets[_]['steadyStateConditions_df']['Qdot'] / triplets[_]['panelSpecs']['AG']
    triplets[_]['df_res']['Qdot/AG'] = triplets[_]['df_res']['Qdot_tube_fluid'] / triplets[_]['panelSpecs']['AG']

In [ ]:
k0 = 3
tab_mat,tab_res,X = nm.find_SK(triplets[k0]['steadyStateConditions_df'],dic_names_exp,columns_list=[1,1,0,1,1,0,1,1,0],options=[],IAM=False,LSM='nnls')
triplets[k0]['steadyStateConditions_df']['Qdot/AG_reg'] = tab_mat @ X[0]
tab_mat,tab_res,X1 = nm.find_SK(triplets[k0]['df_res'],dic_names_num,columns_list=[1,1,0,1,1,0,1,1,0],options=[],IAM=False,LSM='nnls')
triplets[k0]['df_res']['Qdot/AG_reg'] = tab_mat @ X1[0]
print(X[0])
print(X1[0])

In [530]:
# Create a dataframe with X[0] list horizontally
coeff_df_temp = pd.DataFrame([X[0], X1[0]], columns=[f'param_{i}' for i in range(len(X[0]))])

In [531]:
coeff_df = pd.concat([coeff_df, coeff_df_temp], axis=0)

In [ ]:
# Define the list of strings
design_labels = ['S-S\\&T-NI', 'S-S\\&T-NI', 'S-S\\&T-I', 'S-S\\&T-I']
design_labels = [label for label in design_labels for _ in range(2)]

# Insert the column in the first position
coeff_df.insert(0, 'Design', design_labels)

coeff_df

In [ ]:
coeff_df

In [ ]:
# Transform the last 4 columns to strings and replace '0', '0.0', or '0.000' with '-'
coeff_df.iloc[:, -4:] = coeff_df.iloc[:, -4:].applymap(lambda x: '-' if x in [0] else str(x))

coeff_df

In [557]:
latex_table = coeff_df.to_latex(index=False,
                                # escape=False,
                                # column_format='lcccccccccccc'
                                )

In [ ]:
print(coeff_df.to_latex(index=False))

In [559]:
from IPython.display import display, Latex

In [ ]:
latex_table = latex_table.replace("\\begin{tabular}", "\\resizebox{\\textwidth}{!}{\\begin{tabular}")  # Full text width
latex_table = latex_table.replace("\\end{tabular}", "\\end{tabular}}")  # Close the resizebox

# Adding custom lines (1pt and 2pt thick for darker separation)
latex_table = latex_table.replace("\\toprule", "\\hline")  # Replace top border
latex_table = latex_table.replace("\\bottomrule", "\\hline")  # Replace bottom border
latex_table = latex_table.replace("\\midrule", "\\hline")  # Replace midrule
latex_table = latex_table.replace("\\hline", "\\hline\n")  # Ensure spacing around lines

# Insert thicker (2pt) lines every two rows
rows = latex_table.split("\n")
processed_rows = []
row_count = 0
for row in rows:
    processed_rows.append(row)
    if "\\\\" in row:  # Detect the end of a data row
        row_count += 1
        if row_count % 2 == 0:
            processed_rows.append("\\arrayrulewidth=2pt\\hline\\arrayrulewidth=1pt")  # Thicker line after every two rows

# Rebuild the table with processed rows
latex_table = "\n".join(processed_rows)

# Display the final LaTeX code
print(latex_table)

In [ ]:
Latex(latex_table)

In [ ]:
print(coeff_df.to_latex(index=False))

In [550]:
coeff_df.insert(1, 'Test', ['SK', 'SK', 'dark', 'dark']*2)

In [552]:
coeff_df.insert(2, 'Data', ['Experiment', 'Model']*4)

In [538]:
coeff_df['\\eta_{0,b-indoor}'] = coeff_df['\\eta_{0,b-indoor}'].round(3)

In [ ]:
coeff_df

In [541]:
coeff_df['a1 '] = coeff_df['a1 '].round(1)
coeff_df['a3'] = coeff_df['a3'].round(1)
coeff_df['a6'] = coeff_df['a6'].round(3)

In [536]:
coeff_df.drop(columns=[f'param_{i}' for i in [2, 4, 5, 7, 8]], inplace=True)

In [534]:
coeff_df.rename(columns={'param_0' : '\eta_{0,b-indoor}',
                         'param_1' : 'a1',
                         'param_3' : 'a3',
                         'param_6': 'a6'}, inplace=True)

In [ ]:
tab_res

In [ ]:
X

In [ ]:
X1

In [ ]:
plt.scatter(triplets[k0]['df_res']['T_m - T_amb'], triplets[k0]['df_res']['Qdot_tube_fluid']/triplets[k0]['panelSpecs']['AG'])
for i, (u_min, u_max) in enumerate(u_range):
    df_temp = triplets[k0]['df_res'].loc[(u_min <= triplets[k0]['df_res']['u']) & (triplets[k0]['df_res']['u'] < u_max)].copy()
    plt.plot(df_temp['T_m - T_amb'], df_temp['Qdot/AG_reg'])

plt.scatter(triplets[k0]['steadyStateConditions_df']['T_m - T_amb'], triplets[k0]['steadyStateConditions_df']['Qdot']/triplets[k0]['panelSpecs']['AG'])
for i, (u_min, u_max) in enumerate(u_range):
    df_temp = triplets[k0]['steadyStateConditions_df'].loc[(u_min <= triplets[k0]['steadyStateConditions_df']['u']) & (triplets[k0]['steadyStateConditions_df']['u'] < u_max)].copy()
    plt.plot(df_temp['T_m - T_amb'], df_temp['Qdot/AG_reg'])

In [ ]:
print(sklearn.metrics.mean_absolute_error(triplets[k0]['df_res']['Qdot/AG'], triplets[k0]['steadyStateConditions_df']['Qdot/AG']))

In [ ]:
plt.scatter(triplets[0]['steadyStateConditions_df']['Qdot/AG'], triplets[0]['df_res']['Qdot/AG'])
plt.plot(np.arange(0, 350, 1), np.arange(0,350, 1), color='red')
plt.grid()

In [ ]:
triplets[0]['df_res'][['Qdot_experimental']

In [ ]:
triplets[0]['steadyStateConditions_df']['Qdot']